In [12]:
import time
from transformers import pipeline
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import string
from time import strftime, sleep
from tkinter import*
import tkinter as tk
from tkinter.ttk import Progressbar
from PIL import Image, ImageTk, ImageDraw
from tkinter import messagebox
from datetime import datetime
from time import strftime
from pytz import timezone
from gtts import gTTS
import random
from IPython.display import Audio
import pygame
pygame.mixer.init()
root = tk.Tk()
root.geometry("1650x1000")
root['bg'] = 'lightblue'
root.title("Youtube-Transcript Summarizer")

model = MBartForConditionalGeneration.from_pretrained(
    "facebook/mbart-large-50-one-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained(
    "facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

#------------------------- Time -------------------------------

def current_time():
    now_utc = datetime.now(timezone('UTC'))
    now_asia = now_utc.astimezone(timezone('Asia/Kolkata'))
    string = now_asia.strftime('%H:%M:%S %p')
    lbl.config(text=string)
    lbl.after(1000, current_time)


lbl = Label(root, font=(SUNKEN, 15, 'bold'), fg='black',bg='lightblue', border=0, borderwidth=0)
lbl.place(x=1, y=5)
current_time()

#--------------------------- Heading ---------------------------

img = Image.open(r"images\youtube_img.jpg")
img = img.resize((250, 65), Image.ANTIALIAS)
img = ImageTk.PhotoImage(img)

hd_img1 = Label(root, image=img)
hd_img1.place(x=250, y=4.5)
hd_text = Label(root, text="Transcript Summarizer", font=(
    SUNKEN, 38, 'bold'), fg='black', bg='lightblue')
hd_text.place(x=500, y=4.5)

#--------------------- Exit Button ------------------------------

def exit_fun():
    root.destroy()

def iExit():
    iExit_t = messagebox.askyesno("Exit", "Do you want to exit")
    if iExit_t > 0:
        root.destroy()
    else:
        return

exit_btn = Button(root, text='Quit', bg='black', command=iExit, font=('Arial', 13, 'bold'), fg='white',
    activebackground='black', activeforeground='white', border=0, borderwidth=3, cursor='hand2', relief=RIDGE)
exit_btn.place(x=1150, y=10, width=100, height=25)

#----------------------------- Get Link -----------------------------

def getlink():
    link = StringVar()

    def var_text():
        name = link.get()
        with open("link.txt", "w") as text_file:
            text_file.write(name)
        link.set("")

        progress = Progressbar(root, orient=HORIZONTAL,length=200, mode='determinate')

        def youtube():
            progress['value'] = 40
            root.update_idletasks()
            time.sleep(0.5)
            myfile = open('link.txt', "r")
            you_link = myfile.readline()
            myfile.close()
            video_id = you_link.split("=")[1]
            transcript = YouTubeTranscriptApi.get_transcript(video_id)
            full_transcript = ""
            for x in transcript:
                full_transcript += " " + x['text']

            print(len(full_transcript))

            progress['value'] = 80
            root.update_idletasks()
            time.sleep(0.5)

            summerizar = pipeline("summarization")
            itr = int(len(full_transcript)/1000)
            summary_text = []

            progress['value'] = 120
            root.update_idletasks()
            time.sleep(0.1)

            for i in range(0, itr+1):
                start = 0
                start = i*1000

                end = (i+1)*1000
                out = summerizar(full_transcript[start:end],max_length=90)
                out = out[0]
                out = out["summary_text"]
                summary_text.append(out)
            print(summary_text)

            progress['value'] = 160
            root.update_idletasks()
            time.sleep(0.1)

            with open("summary.txt", "w") as text_file:
                text_file.write(summary_text[0])

            progress['value'] = 200
            root.update_idletasks()
            time.sleep(5)
        progress.place(x=930, y=245)
        transcribe = Button(root, text='Transcribe', bg='black', command=youtube, font=('Arial', 13, 'bold'), fg='white',
            activebackground='black', activeforeground='white', border=0, borderwidth=3, cursor='hand2', relief=RIDGE)
        transcribe.place(x=800, y=240, width=137, height=32)

    link_lbl = Label(root, text="Enter URL", bg='black',fg='lightblue', font=('SUNKEN', 13, 'bold'))
    link_lbl.place(x=100, y=250)
    link_data = tk.Entry(root, width=20, textvariable=link,font=('SUNKEN', 13, 'normal'))
    link_data.place(x=200, y=250)

    sub_btn = Button(root, text="Submit", command=var_text, bg='white', fg='black',
        border=1, borderwidth=3, font=('Arial', 10), activebackground='black',activeforeground='white')
    sub_btn.place(x=400, y=250, height=25)


start_img = Image.open(r"images\activate.png")
start_img = start_img.resize((150, 35), Image.ANTIALIAS)
start_img = ImageTk.PhotoImage(start_img)
start = Button(root, command=getlink, image=start_img,cursor='hand2',bg='lightblue', borderwidth=4, activebackground='black')
start.place(x=480, y=100, width=150, height=45)


#------------------------------ Textarea ------------------------------

txtarea = Text(root, wrap=WORD, bg='white', font=("TimesNewRoman", 11), width=70,height=15, borderwidth=4, relief='solid', padx=10, pady=10, spacing2=5)
txtarea.pack(pady=100)
txtarea.place(x=280, y=300)


#-------------------------------- Translate ------------------------------

def english():
    txtarea.delete('1.0', END)
    file = open('summary.txt', "r")
    data = file.readline()
    file.close()
    txtarea.delete('1.0', END)
    txtarea.insert(tk.END, data)

    pygame.mixer.init()
    r1 = random.randint(1, 10000000)
    r2 = random.randint(1, 10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) + ".mp3"
    file = open('summary.txt', "r")
    data = file.readline()
    file.close()
    text = data
    language = 'en'
    speech = gTTS(text=text, lang=language, slow=False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)

    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)

    play = Button(root, text="Play", command=play_sound, bg='black', fg='green', cursor='hand2',
        border=2, borderwidth=2, activebackground='black', font=('SUNKEN', 13, 'bold'))
    play.place(x=220, y=310, width=40, height=20)


def hindi():
    txtarea.delete('1.0', END)
    file = open('summary.txt', "r")
    data = file.readline()
    file.close()
    model_inputs = tokenizer(data, return_tensors="pt")
    # translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
    )
    translation = tokenizer.batch_decode(
        generated_tokens, skip_special_tokens=True)
    print(translation)

    txtarea.insert(tk.END, translation[0])
    text = translation[0]
    pygame.mixer.init()
    r1 = random.randint(1, 10000000)
    r2 = random.randint(1, 10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) + ".mp3"
    language = 'hi'
    speech = gTTS(text=text, lang=language, slow=False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)

    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)

    play = Button(root, text="Play", command=play_sound, bg='black', fg='green', cursor='hand2',
        border=2, borderwidth=2, activebackground='black', font=('SUNKEN', 13, 'bold'))
    play.place(x=220, y=410, width=40, height=20)


def german():
    txtarea.delete('1.0', END)
    file = open('summary.txt', "r")
    data = file.readline()
    file.close()
    model_inputs = tokenizer(data, return_tensors="pt")
    # translate from English to German
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["de_DE"]
    )
    translation = tokenizer.batch_decode(
        generated_tokens, skip_special_tokens=True)
    print(translation)
    txtarea.insert(tk.END, translation[0])
    text = translation[0]
    pygame.mixer.init()
    r1 = random.randint(1, 10000000)
    r2 = random.randint(1, 10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) + ".mp3"
    language = 'de'
    speech = gTTS(text=text, lang=language, slow=False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)

    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)

    play = Button(root, text="Play", command=play_sound, bg='black', fg='green', cursor='hand2',
        border=2, borderwidth=2, activebackground='black', font=('SUNKEN', 13, 'bold'))
    play.place(x=220, y=510, width=40, height=20)


def french():
    txtarea.delete('1.0', END)
    file = open('summary.txt', "r")
    data = file.readline()
    file.close()
    model_inputs = tokenizer(data, return_tensors="pt")
    # translate from English to French
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["fr_XX"]
    )
    translation = tokenizer.batch_decode(
        generated_tokens, skip_special_tokens=True)
    print(translation)
    txtarea.insert(tk.END, translation[0])
    pygame.mixer.init()
    r1 = random.randint(1, 10000000)
    r2 = random.randint(1, 10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) + ".mp3"
    text = translation[0]
    language = 'fr'
    speech = gTTS(text=text, lang=language, slow=False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)

    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)

    play = Button(root, text="Play", command=play_sound, bg='black', fg='green', cursor='hand2', 
        border=2, borderwidth=2, activebackground='black', font=('SUNKEN', 13, 'bold'))
    play.place(x=220, y=610, width=40, height=20)


def bengali():
    txtarea.delete('1.0', END)
    file = open('summary.txt', "r")
    data = file.readline()
    file.close()
    model_inputs = tokenizer(data, return_tensors="pt")
    # translate from English to Bengali
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["bn_IN"]
    )
    translation = tokenizer.batch_decode(
        generated_tokens, skip_special_tokens=True)
    print(translation)
    txtarea.insert(tk.END, translation[0])
    text = translation[0]
    pygame.mixer.init()
    r1 = random.randint(1, 10000000)
    r2 = random.randint(1, 10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) + ".mp3"
    language = 'bn'
    speech = gTTS(text=text, lang=language, slow=False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)

    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)
    play = Button(root, text="Play", command=play_sound, bg='black', fg='green', cursor='hand2',
        border=2, borderwidth=2, activebackground='black', font=('SUNKEN', 13, 'bold'))
    play.place(x=1090, y=310, width=40, height=20)


def chinese():
    txtarea.delete('1.0', END)
    file = open('summary.txt', "r")
    data = file.readline()
    file.close()
    model_inputs = tokenizer(data, return_tensors="pt")
    # translate from English to Chinese
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"]
    )
    translation = tokenizer.batch_decode(
        generated_tokens, skip_special_tokens=True)
    print(translation)
    txtarea.insert(tk.END, translation[0])
    text = translation[0]
    pygame.mixer.init()
    r1 = random.randint(1, 10000000)
    r2 = random.randint(1, 10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) + ".mp3"
    language = 'zh-cn'
    speech = gTTS(text=text, lang=language, slow=False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)

    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)
    play = Button(root, text="Play", command=play_sound, bg='black', fg='green', cursor='hand2',
        border=2, borderwidth=2, activebackground='black', font=('SUNKEN', 13, 'bold'))
    play.place(x=1090, y=410, width=40, height=20)


def tamil():
    txtarea.delete('1.0', END)
    file = open('summary.txt', "r")
    data = file.readline()
    file.close()
    model_inputs = tokenizer(data, return_tensors="pt")
    # translate from English to Tamil
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["ta_IN"]
    )
    translation = tokenizer.batch_decode(
        generated_tokens, skip_special_tokens=True)
    print(translation)
    txtarea.insert(tk.END, translation[0])
    text = translation[0]
    pygame.mixer.init()
    r1 = random.randint(1, 10000000)
    r2 = random.randint(1, 10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) + ".mp3"
    language = 'ta'
    speech = gTTS(text=text, lang=language, slow=False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)

    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)
    play = Button(root, text="Play", command=play_sound, bg='black', fg='green', cursor='hand2',
        border=2, borderwidth=2, activebackground='black', font=('SUNKEN', 13, 'bold'))
    play.place(x=1090, y=510, width=40, height=20)


def telugu():
    txtarea.delete('1.0', END)
    file = open('summary.txt', "r")
    data = file.readline()
    file.close()
    model_inputs = tokenizer(data, return_tensors="pt")
    # translate from English to Telugu
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["te_IN"]
    )
    translation = tokenizer.batch_decode(
        generated_tokens, skip_special_tokens=True)
    print(translation)
    txtarea.insert(tk.END, translation[0])
    text = translation[0]
    pygame.mixer.init()
    r1 = random.randint(1, 10000000)
    r2 = random.randint(1, 10000000)
    sound_file = str(r2)+"SUMMARY"+str(r1) + ".mp3"
    language = 'te'
    speech = gTTS(text=text, lang=language, slow=False,)
    speech.save(sound_file)
    Audio(sound_file, autoplay=False)

    def play_sound():
        pygame.mixer.music.load(sound_file)
        pygame.mixer.music.play(loops=0)
    play = Button(root, text="Play", command=play_sound, bg='black', fg='green', cursor='hand2',
        border=2, borderwidth=2, activebackground='black', font=('SUNKEN', 13, 'bold'))
    play.place(x=1090, y=610, width=40, height=20)


def stop_sound():
    pygame.mixer.music.stop()


stop_img = Image.open(r"images\stop_button.jpg")
stop_img = stop_img.resize((40, 38), Image.ANTIALIAS)
stop_img = ImageTk.PhotoImage(stop_img)
stop = Button(root, image=stop_img, command=stop_sound,cursor='hand2', border=0, borderwidth=0, activebackground='lightblue')
stop.place(x=550, y=600, width=50, height=40)


#-------------------------------- Language-button ------------------------------

btn_eng = Image.open(r"images\button_english.png")
btn_eng = btn_eng.resize((100, 25), Image.ANTIALIAS)
btn_eng = ImageTk.PhotoImage(btn_eng)
eng = Button(root, image=btn_eng, command=english, fg='blue',bg='lightblue',cursor='hand2',border=0,borderwidth=0,activebackground='lightblue')
eng.place(x=80, y=300, width=135, height=37)

btn_hindi = Image.open(r"images\button_hindi.png")
btn_hindi = btn_hindi.resize((100, 25), Image.ANTIALIAS)
btn_hindi = ImageTk.PhotoImage(btn_hindi)
hindi = Button(root, image=btn_hindi, command=hindi, fg='blue',bg='lightblue', cursor='hand2',border=0,borderwidth=0,activebackground='lightblue')
hindi.place(x=80, y=400, width=135, height=37)

btn_german = Image.open(r"images\button_german.png")
btn_german = btn_german.resize((100, 25), Image.ANTIALIAS)
btn_german = ImageTk.PhotoImage(btn_german)
german = Button(root, image=btn_german, command=german, fg='blue', bg='lightblue',cursor='hand2',border=0,borderwidth=0,activebackground='lightblue')
german.place(x=80, y=500, width=135, height=37)

btn_french = Image.open(r"images\button_french.png")
btn_french = btn_french.resize((100, 25), Image.ANTIALIAS)
btn_french = ImageTk.PhotoImage(btn_french)
french = Button(root, image=btn_french, command=french, fg='blue', bg='lightblue',cursor='hand2', border=0, borderwidth=0, activebackground='lightblue')
french.place(x=80, y=600, width=135, height=37)

btn_bengali = Image.open(r"images\button_bengali.png")
btn_bengali = btn_bengali.resize((100, 25), Image.ANTIALIAS)
btn_bengali = ImageTk.PhotoImage(btn_bengali)
bengali = Button(root, image=btn_bengali, command=bengali, fg='blue', bg='lightblue',cursor='hand2', border=0, borderwidth=0, activebackground='lightblue')
bengali.place(x=950, y=300, width=135, height=37)

btn_chinise = Image.open(r"images\button_chinese.png")
btn_chinise = btn_chinise.resize((100, 25), Image.ANTIALIAS)
btn_chinise = ImageTk.PhotoImage(btn_chinise)
chinise = Button(root, image=btn_chinise, command=chinese, fg='blue',bg='lightblue',cursor='hand2',border=0,borderwidth=0,activebackground='lightblue')
chinise.place(x=950, y=400, width=135, height=37)

btn_tamil = Image.open(r"images\button_tamil.png")
btn_tamil = btn_tamil.resize((100, 25), Image.ANTIALIAS)
btn_tamil = ImageTk.PhotoImage(btn_tamil)
tamil = Button(root, image=btn_tamil, command=tamil,fg='blue', bg='lightblue', cursor='hand2',border=0,borderwidth=0,activebackground='lightblue')
tamil.place(x=950, y=500, width=135, height=37)

btn_telugu = Image.open(r"images\button_telugu.png")
btn_telugu = btn_telugu.resize((100, 25), Image.ANTIALIAS)
btn_telugu = ImageTk.PhotoImage(btn_telugu)
telugu = Button(root, image=btn_telugu, command=telugu, fg='blue', bg='lightblue',cursor='hand2', border=0, borderwidth=0, activebackground='lightblue')
telugu.place(x=950, y=600, width=135, height=37)


root.mainloop()


10413


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


[" YouTube star says he finally cracked Google but this journey of mine was quite different from the generic Google interview experiences that you're used to seeing on YouTube so this journey with a lot of apps and flows ups and downs took me around one year right from the recruiter reaching out to me to me finally getting that offer .", " In 2020 I interviewed for L3 that was the sd1 rule and uska experience when the last video may already share for a very close call but this time I got into Google for that sd2 that is L4 rule that is one level up the L3 . 2022 Amazon I was actually looking for a switch but I didn't want to interview for Google because just that the parameters are kind of more stricter in L4 experience just", ' Google is definitely a great company to work for but again these things are also priority for me I guess so as a result I was not replying much to the Google recruiters back then and I was like interviewing for other companies and I cracked a couple of offers f

c:\Users\Siddhi Kakkar\anaconda3\lib\site-packages\transformers\generation\utils.py:1369: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['यूट्यूब स्टार कहता है कि उसने अंततः गूगल को उखाड़ दिया लेकिन मेरी यह यात्रा सामान्य गूगल साक्षात्कार अनुभवों से काफी भिन्न थी जो आप यूट्यूब पर देखने के लिए प्रयुक्त हैं तो यह यात्रा बहुत से एप्लिकेशनों और प्रवाहों और उतार-चढ़ावों के साथ मुझे एक साल के आसपास लगा था भर्तीकर्ता से अंततः मुझे उस प्रस्ताव को प्राप्त करने के लिए मेरे पास पहुंचने के लिए।']


...

['YouTube明星说他终于打败了谷歌,但我的这一旅程与你习惯于在YouTube上看到的一般谷歌面试体验截然不同,因此这趟许多应用程序和流程的旅程花了我大约一年,从招聘人员接触到我最后得到了那份报价。']


c:\Users\Siddhi Kakkar\anaconda3\lib\site-packages\transformers\generation\utils.py:1369: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['யூடியூப் நட்சத்திரம், இறுதியில் அவர் கூகுலை வெடித்தார் என்று கூறுகிறார், ஆனால் இந்த என்னுடைய பயணமானது, நீங்கள் YouTube ல் காணும் பொதுவான கூகுள் நேர்காணல் அனுபவங்களிலிருந்து முற்றிலும் வேறுபட்டது. எனவே, இந்த பயணமானது, பல செயலிகளுடன், பெருகுதல்கள், குறைவுகள் ஆகியவை என்னை ஒரு வருடத்திற்குள் பிடித்தன. நியமிப்பாளர் என்னைத் தொடர்பு கொண்டு இறுதியில் அந்த அளிப்பை எனக்குக் கொடுத்தார்.']


c:\Users\Siddhi Kakkar\anaconda3\lib\site-packages\transformers\generation\utils.py:1369: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['YouTube నక్షత్రము అతను చివరకు గోల్ పగుళ్లు చెప్పారు కానీ గని ఈ ప్రయాణం నేను చాలా వివిధ ఉంది రుతుదారిక గోల్ انٹرویو అనుభవం, మీరు YouTube న చూడటానికి ఉపయోగించారు కనుక ఈ ప్రయాణం నేను చాలాאפליקציות మరియు ఫ్లోస్ అప్లు మరియు డౌన్లు తో ఒక సంవత్సరం కుడి Rekrutateur నాకు చేరట నుండి నాకు చివరకు ఆ понуда పొందుటకు తీసుకోవాలని.']
